In [1]:
%load_ext autoreload

In [2]:
import sys
import copy
sys.path.append('..')

import numpy as np
import tensorflow as tf

from lib.cube import Cube
from lib.solver import CubeSolver
from lib.models import CNN

%autoreload

In [3]:
config = {}


config['model_params'] = {'embed_dim':100,
                          'num_filters':50,
                          'num_conv_layers':3,
                          'kernel_size':2,
                          'regularization_constant':.5,
                          'num_dense_layers':3,
                           'dense_activation':'elu'}
#global vars
solver = CubeSolver()
solver.model = CNN(**config['model_params'])
solver.load_model_weights('../models/base_model_v2_20190404001914/weights')

c = 0.1 #exploration hyper parameter
v = 0.1 # virtual loss hyper parameter
sovled_cube = Cube() 

In [4]:

class Node(object):
    def __init__(self,state, parent=None):
        self.parent=parent
        self.children=[]
        self.state=copy.deepcopy(state)
        self.action_taken_string=None
        self.N = self._init_N()
        self.W = self._init_W()
        self.L = self._init_L()
        self.P = self._init_P()
        
    def _init_N(self):
        N = dict(zip(cube_moves,[0]*len(cube_moves)))
        return N
    def _init_W(self):
        W = dict(zip(cube_moves,[0]*len(cube_moves)))
        return W
    
    def _init_L(self):
        L = dict(zip(cube_moves,[0]*len(cube_moves)))
        return L
    
    def _init_P(self):
        node_state = copy.deepcopy(self.state)
        node_state = tf.convert_to_tensor(node_state)
        node_state = tf.expand_dims(node_state, 0)
        action_probs = tf.keras.activations.softmax(solver.model(node_state)).numpy()[0]
        P = dict(zip(cube_moves,action_probs))
        return P
    
    def get_Q_st(self):
        W_st = np.array(list(self.W.values()))
        L_st = np.array(list(self.L.values()))
        Q_st = W_st - L_st
        return Q_st
    
    def get_U_st(self):
        P_st = np.array(list(self.P.values()))
        N_st = np.array(list(self.N.values()))
        U_st = c*P_st*np.sqrt(N_st.sum()) / (1 + N_st)
        return U_st
    
    def update_memory(self, q_current_state):
        self.W[self.action_taken_string] = max(self.W[self.action_taken_string],q_current_state)
        self.N[self.action_taken_string] += 1
#         self.L[self.action_taken_string] -= v
        self.action_taken_string=None


In [9]:
solve_rate = 0
c = .1 #exploration hyper parameter
v = .5 # virtual loss hyper parameter
for trial in range(10):


    cube = Cube() #global var
    cube.shuffle(num_shuffles=5,verbose=False)\
#     print("down_p")
#     cube.down_p()
#     print("right")
#     cube.right()
#     print("front")
#     cube.front()
#     print('-----------')


    cube_moves=[] #global var
    for move in cube.func_list:
        cube_moves.append(str(move).split()[2].split('.')[1])
    root = Node(cube.state)


    for i in range(1000): #Should this be time based?
        #selection
        current_node = root
        while current_node.children: #while not leaf node
            Q_st = current_node.get_Q_st()
            U_st = current_node.get_U_st()
            A_st = np.argmax(U_st + Q_st)
            A_st_string = cube_moves[A_st]
#             print(f"Enter Selection: {A_st_string}")


            #update loss
            current_node.L[A_st_string] += v
            #save action taken
            current_node.action_taken_string=A_st_string
            #move to next node
            current_node = current_node.children[A_st]


        #check for termination
        if (current_node.state == sovled_cube.state).all():
#             print("Cube is solved")
            solve_rate+=1
            break



        #expansion
#         print("Expansion")
        for move in cube.func_list:
            cube.set_state(current_node.state)
            move()
            move_name = str(move).split()[2].split('.')[1] #could be inefficient
            new_state = Node(cube.state, parent=current_node)
            current_node.children.append(new_state)

        #simulation ?
#         print("Simulation")

        current_state = copy.deepcopy(current_node.state)
        current_state = tf.convert_to_tensor(current_state)
        current_state = tf.expand_dims(current_state, 0)

        a_current_state = solver.model(current_state).numpy()[0].argmax()
        current_node.action_taken_string=cube_moves[a_current_state]

        q_current_state = solver.model(current_state).numpy()[0].max() #are q and v the same?


        #backpropagation
#         print("Backpropagation")
        current_node.update_memory(q_current_state)
        while current_node.parent is not None:
#             print("Propagating")
            current_node = current_node.parent
            current_node.update_memory(q_current_state)


#         if i == 99:
#             print("Time Out")
#         else:
#             print('--------------')
solve_rate

4

In [44]:
bn = tf.keras.layers.BatchNormalization(1)

x = tf.constant([[3.,7.,2.,4.],[.1,.7,.8,.9]])

fc1 = tf.keras.layers.Dense(10)
print(fc1(x).shape)
out = bn(fc1(x),training=True)
print(out)
out.numpy().mean(),out.numpy().std()

(2, 10)
tf.Tensor(
[[ 0.99546456  0.9990916   0.9998145  -0.9992658  -0.9998772  -0.99897933
  -0.99988675  0.99983263  0.9998877   0.99891365]
 [-0.9954648  -0.9990915  -0.99981445  0.9992658   0.9998771   0.9989795
   0.99988663 -0.99983263 -0.9998877  -0.99891394]], shape=(2, 10), dtype=float32)


(-2.0861625e-08, 0.9991022)

In [49]:
(3 - 1.55) / 1.45

1.0

In [50]:
(.1 - 1.55) / 1.45

-1.0

In [46]:
(3+.1) /2

1.55

In [48]:
np.std([3,.1])

1.45

In [40]:
fc = tf.keras.Sequential()
fc_layer = tf.keras.layers.Dense(2,activation=None)
batch_norm = tf.keras.layers.BatchNormalization(axis=0)
activation = tf.keras.layers.ELU()

fc.add(fc_layer)
fc.add(batch_norm)
fc.add(activation)

In [39]:
print(fc(x, training=False))
print(fc(tf.expand_dims(tf.constant([3.,7.,2.,4.]),0), training= True))

tf.Tensor(
[[-0.99957955 -0.9800308 ]
 [-0.15352827 -0.575464  ]], shape=(2, 2), dtype=float32)
tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)


In [62]:
x = tf.expand_dims(tf.constant([[3.,7.],[.1,.7]]),0)
x = tf.expand_dims(x,0)
x.shape

TensorShape([1, 1, 2, 2])

In [65]:
embedding = tf.keras.layers.Embedding(27, 100)
cnn_layer = tf.keras.layers.Conv3D(filters=50,
                                   kernel_size=2,
                                   data_format='channels_last',
                                   padding = 'valid')

bn = tf.keras.layers.BatchNormalization(axis = -1)

print(current_state.shape)
print(embedding(current_state).shape)
print(cnn_layer(embedding(current_state)).shape)
out = cnn_layer(embedding(current_state))
normed_out = bn(out, training= True)
fm = 19
print(out.numpy()[0,:,:,:,fm].shape)
print(out.numpy()[0,:,:,:,fm].mean(), out.numpy()[:,:,:,:,fm].std())
print(normed_out.numpy()[0,:,:,:,fm].mean(), normed_out.numpy()[0,:,:,:,fm].std())


(1, 3, 3, 3)
(1, 3, 3, 3, 100)
(1, 2, 2, 2, 50)
(2, 2, 2)
-0.0093600545 0.023751186
-1.4901161e-08 0.6005517


In [10]:
flatten = tf.keras.layers.Flatten()
#         self.fc = self._build_fc_layers(num_dense_layers,
#                                         activation=self.dense_activation)
#         self.output_layer = tf.keras.layers.Dense(12,
#                                             activation=self.dense_activation,
#                                             kernel_regularizer=tf.keras.regularizers.l2(l=self.rc),
#                                             bias_regularizer=tf.keras.regularizers.l2(l=self.rc))

In [13]:
flatten(out).shape
output_layer = tf.keras.layers.Dense(12)
print(flatten(out).shape)
output_layer(flatten(out)).shape

(1, 400)


TensorShape([1, 12])

In [ ]:
self.embed_dim = embed_dim
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.rc = regularization_constant
        self.dense_activation = dense_activation
        self.embedding = tf.keras.layers.Embedding(27, self.embed_dim)
        self.cnn = self._build_conv_layers(num_conv_layers)
        self.flatten = tf.keras.layers.Flatten()
        self.fc = self._build_fc_layers(num_dense_layers)
        self.output_layer = tf.keras.layers.Dense(12,
                                            activation=self.dense_activation,
                                            kernel_regularizer=tf.keras.regularizers.l2(l=self.rc),
                                            bias_regularizer=tf.keras.regularizers.l2(l=self.rc))

    def _build_conv_layers(self, num_conv_layers):
        '''
        helper function for init
        '''
        cnn = tf.keras.Sequential()
        for i in range(num_conv_layers):
            if i == 0:
                padding = 'valid'
            else:
                padding = 'same'
            cnn_layer = tf.keras.layers.Conv3D(filters=self.num_filters,
                                               kernel_size=self.kernel_size,
                                               data_format='channels_last',
                                               padding = padding)
            cnn.add(cnn_layer)
        return cnn


    def _build_fc_layers(self, num_dense_layers):
        '''
        helper function for init
        '''
        fc = tf.keras.Sequential()
        for i in range(num_dense_layers):
            fc_layer = tf.keras.layers.Dense(50,
                                             kernel_regularizer=tf.keras.regularizers.l2(l=self.rc),
                                             bias_regularizer=tf.keras.regularizers.l2(l=self.rc),
                                             activation=self.dense_activation)
            fc.add(fc_layer)
        return fc


In [ ]:
current_node = root
while current_node.children: #while not leaf node
    Q_st = current_node.get_Q_st()
    U_st = current_node.get_U_st()
    A_st = np.argmax(U_st + Q_st)
    A_st_string = cube_moves[A_st]
    print(A_st_string)
    #update loss
    current_node.L[A_st_string] += v
    #move to next node
    current_node = current_node.children[A_st]
    

In [ ]:
root.state

In [ ]:
current_node.state

In [ ]:
current_node.parent.state